<a href="https://colab.research.google.com/github/LeeYuuuan/Graph-Kernel-Isomorphic-/blob/main/classification_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%pip install grakel
import grakel
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import networkx.algorithms.isomorphism as iso
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics, svm
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
from grakel.kernels import WeisfeilerLehman, VertexHistogram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
def draw_graphs(G,G1):
    pos = nx.spring_layout(G)
    pos1 = nx.spring_layout(G1) 
    plt.subplot(121)
    nx.draw(G, pos, with_labels=True, node_color='pink', node_size=500)
    plt.subplot(122)
    nx.draw(G1, pos1, with_labels=True, node_size=500)
    plt.show()


In [2]:
def GetGraphData(dataset_name):
  dataset = grakel.datasets.fetch_dataset(dataset_name, verbose=True, data_home=None, download_if_missing=True, with_classes=True, produce_labels_nodes=False, prefer_attr_nodes=False, prefer_attr_edges=False, as_graphs=False)
  G = dataset.data
  y = dataset.target
  return G, y

In [17]:
def Classification(dataset_name, kernel_name):
    G, y = GetGraphData(dataset_name)
    G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.1, random_state=42)

    C_grid = (10. ** np.arange(-4,6,1) / len(G)).tolist()
    if kernel_name == "weisfeiler_lehman":

        gk = WeisfeilerLehman(n_iter=4, base_graph_kernel=VertexHistogram, normalize=True)
        K_train = gk.fit_transform(G_train)
        K_test = gk.transform(G_test)

        

        clf = SVC(kernel="precomputed")
        clf.fit(K_train, y_train)
        y_pred = clf.predict(K_test)
        
        print(classification_report(y_test, y_pred))
        print(str(round(accuracy_score(y_test, y_pred)*100, 2)), "%")
    
    if kernel_name == "wl":
        estimator = make_pipeline(
            grakel.kernels.weisfeiler_lehman(normalize=True),
            GridSearchCV(SVC(kernel='precomputed'), dict(C=C_grid),
                         scoring='accuracy', cv=10))
        n_folds = 10
        cvp = cross_val_predict(estimator, G, y, cv=n_folds)
        acc = accuracy_score(y, cvp)
        print("Accuracy:", str(round(acc*100, 2)) + "%")
        print(classification_report(y,cvp))
      
    
    if kernel_name == "ShortestPath":
        estimator = make_pipeline(
            grakel.kernels.ShortestPath(normalize=True),
            GridSearchCV(SVC(kernel='precomputed'), dict(C=C_grid),
                        scoring='accuracy', cv=10))
        
        n_folds = 10
        cvp = cross_val_predict(estimator, G, y, cv=n_folds)
        acc = accuracy_score(y, cvp)
        print("Accuracy:", str(round(acc*100, 2)) + "%")
        print(classification_report(y,cvp))

        # sns.heatmap(cm,cmap = sns.color_palette("Blues"),annot = True

In [19]:
t0 = time.time()
Classification("BZR", "weisfeiler_lehman")
print("done in %0.3fs\n" % (time.time() - t0))

Extracting dataset  BZR..
Parsing dataset  BZR..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
              precision    recall  f1-score   support

          -1       0.85      1.00      0.92        35
           1       0.00      0.00      0.00         6

    accuracy                           0.85        41
   macro avg       0.43      0.50      0.46        41
weighted avg       0.73      0.85      0.79        41

85.37 %
done in 0.486s



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
